In [ ]:
import os 
import re
import ast
import anndata as ad 
import numpy as np 
import pandas as pd
import scanpy as sc
import seaborn as sns
from shapely.geometry import Polygon
from tqdm import tqdm

In [ ]:
Z_SLICE = 4.5 # this is the middle slice of the z-stack 

In [ ]:
ddir = '/sbgenomics/project-files/HTAN-HTAPP/Merfish'

In [ ]:
def extract_cell_coords(adata, z_slice = Z_SLICE):
    
    x_centroid = []
    y_centroid = []

    # FIXME - there is almost certainly a better way to do this 
    for i,cell in adata.obs.iterrows():
        
        # find index of middle slice in z-stack 
        middle_slice_index = ast.literal_eval(cell['z_height']).index(Z_SLICE)
    
        # create x,y coords for the boundary of each cell in the middle z-slice 
        coords = list(zip(ast.literal_eval(cell['x_boundary'])[middle_slice_index][0],ast.literal_eval(cell['y_boundary'])[middle_slice_index][0]))
    
        # create a polygon based on the coordinates and extract the cell centroid 
        poly = Polygon(coords)
        x_centroid.append(poly.centroid.x)
        y_centroid.append(poly.centroid.y)
    
    # set min coordinates to zero for convenience and save as new cols in adata.obs 
    x_centroid = np.array(x_centroid)
    y_centroid = np.array(y_centroid)
    adata.obs['x_centroid'] = x_centroid - x_centroid.min()
    adata.obs['y_centroid'] = y_centroid - y_centroid.min()

    return adata

In [ ]:
adatas_dict = {}
for i,fh in tqdm(enumerate(os.listdir(ddir))):

    # read anndata files, save in dict with sample name as key, extract cell coords 
    sample_id = re.split('_',fh)[0]
    adatas_dict[sample_id] = ad.read_h5ad(os.path.join(ddir,fh))
    adatas_dict[sample_id] = extract_cell_coords(adatas_dict[sample_id], z_slice = Z_SLICE)